In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# List of URLs
urls = [
    "https://www.basketball-reference.com/teams/CHI/2024.html",
    "https://www.basketball-reference.com/teams/LAL/2024.html",
    "https://www.basketball-reference.com/teams/BOS/2024.html",
    "https://www.basketball-reference.com/teams/BRK/2024.html",
    "https://www.basketball-reference.com/teams/TOR/2024.html",
    "https://www.basketball-reference.com/teams/PHI/2024.html",
    "https://www.basketball-reference.com/teams/NYK/2024.html",
    "https://www.basketball-reference.com/teams/CLE/2024.html",
    "https://www.basketball-reference.com/teams/DET/2024.html",
    "https://www.basketball-reference.com/teams/IND/2024.html",
    "https://www.basketball-reference.com/teams/MIL/2024.html",
    "https://www.basketball-reference.com/teams/WAS/2024.html",
    "https://www.basketball-reference.com/teams/ATL/2024.html",
    "https://www.basketball-reference.com/teams/ORL/2024.html",
    "https://www.basketball-reference.com/teams/MIA/2024.html",
    "https://www.basketball-reference.com/teams/CHO/2024.html",
    "https://www.basketball-reference.com/teams/MIN/2024.html",
    "https://www.basketball-reference.com/teams/OKC/2024.html",
    "https://www.basketball-reference.com/teams/DEN/2024.html",
    "https://www.basketball-reference.com/teams/POR/2024.html",
    "https://www.basketball-reference.com/teams/UTA/2024.html",
    "https://www.basketball-reference.com/teams/LAC/2024.html",
    "https://www.basketball-reference.com/teams/GSW/2024.html",
    "https://www.basketball-reference.com/teams/PHO/2024.html",
    "https://www.basketball-reference.com/teams/SAC/2024.html",
    "https://www.basketball-reference.com/teams/MEM/2024.html",
    "https://www.basketball-reference.com/teams/NOP/2024.html",
    "https://www.basketball-reference.com/teams/HOU/2024.html",
    "https://www.basketball-reference.com/teams/SAS/2024.html",
    "https://www.basketball-reference.com/teams/DAL/2024.html"
    # Add more team URLs as needed
]

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'

def scrape_stats(url):
    headers = {'User-Agent': user_agent}
    
    logging.info(f"Attempting to scrape data from {url}")
    
    try:
        response = requests.get(url, headers=headers, timeout=30)
        logging.info(f"Received response with status code: {response.status_code}")
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            table = soup.find('table', {'id': 'per_game'})
            
            if table:
                columns_of_interest = {
                    "player": "Player", "g": "G", "trb_per_g": "TRB",
                    "ast_per_g": "AST", "stl_per_g": "STL", "blk_per_g": "BLK",
                    "tov_per_g": "TOV", "pts_per_g": "PTS"
                }
                
                rows = []
                for row in table.tbody.find_all('tr'):
                    row_data = {col_name: row.find('td', {'data-stat': stat}).text.strip() 
                                for stat, col_name in columns_of_interest.items() 
                                if row.find('td', {'data-stat': stat})}
                    if row_data:
                        rows.append(row_data)
                
                df = pd.DataFrame(rows)
                logging.info(f"Successfully extracted data for {len(df)} players.")
                return df
            else:
                logging.warning("Table not found on the webpage.")
                return pd.DataFrame()
        else:
            logging.error(f"Failed to retrieve the webpage. Status code: {response.status_code}")
            return pd.DataFrame()
    except requests.Timeout:
        logging.error("Request timed out after 30 seconds.")
    except Exception as e:
        logging.error(f"An error occurred: {str(e)}")
    
    return pd.DataFrame()

all_data = []

for url in urls:
    df = scrape_stats(url)
    if not df.empty:
        all_data.append(df)
    
    # Wait between 60 to 90 seconds before the next request
    delay = random.uniform(60, 90)
    logging.info(f"Waiting {delay:.2f} seconds before the next request...")
    time.sleep(delay)

if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    print(final_df)
    final_df.to_csv('nba_per_game_stats.csv', index=False)
    logging.info("Data saved to 'nba_per_game_stats.csv'")
else:
    logging.warning("No data was scraped. Please check the logs for errors.")

logging.info("Scraping process completed.")

Scraping data from https://www.basketball-reference.com/teams/CHI/2024.html...


In [2]:
# <div class="table_container tabbed current is_setup" id="div_per_game">	